## **Analytical Questions - What is the overall economic health of Americans?**
1.  **How much liquidity do Americans have?**
2.	What does disposable income look like for Americans over time? What is the comparison between ‘poor’ and ‘wealthy' areas(s)/region(s). Can I normalize the overall dataset with this approach–maybe I can use an overlay of cities and towns, etc..?
3. **What are people’s spending habits? how are people spending over time, more or less?** 
4. Can I use this to help better understand trends like housing, food, healthcare spending, etc. What are Americans doing with their disposable income?
4.	 **Are Americans enjoying economic growth over time? Or are there economic decline in earnings?**
--------------------------------------------------------------------------------------------------------------------
- (JC): How does income correlate with specific components of the cost of living (e.g., housing affordability, food prices)? 
- (JC): Have certain components of the cost of living increased at a faster rate than income in some areas? 
- (JD): How did import/export volumes fluctuate monthly across major East Coast ports in 2024?
- (FL): **How are people spending over time, more or less?**
- (FL): **How much disposable income do Americans have over time?** 
- (FL): **What does the cost of living look like for North Americans?** 
- **Note:** I'll be using one or two of these question in my EDA. And the following questions below 




In [1]:
# Import necessary libraries
from scipy.stats import pearsonr # for correlation calculation
import pandas as pd # for data manipulation using DataFrames
import numpy as np # for numerical operations 

import seaborn as sns # for data visualization 
import matplotlib.pyplot as plt # for data visualization
import matplotlib.ticker as xticks # for axis formatting
import plotly.express as px # for interactive plots
import math # for mathematical operations not found in numpy or native python...